In [104]:
from pymongo import MongoClient
import pprint

import requests

from bs4 import BeautifulSoup

import copy

import pandas as pd

In [3]:
test_url = 'https://www.vegasinsider.com/nfl/matchups/matchups.cfm/week/1/season/2020'
r = requests.get(test_url)

In [4]:
r.status_code

200

In [5]:
r.content

b'<!-- manual generation from poegenerators:poeMatchupsIndexPage Tue Aug 19 8:05 AM-->\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN"> \r\n<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/" lang="en">\r\n<head>\r\n\r\n        <title>NFL Matchups 2020 - Lines, Betting Trends, and ATS results</title>\r\n        <meta http-equiv="x-ua-compatible" \tcontent="IE=edge">\r\n        <meta name="Keywords" \t\t\t\tcontent="nfl matchups,football matchups,pro football matchups">\r\n        <meta name="Description" \t\t\tcontent="Get top NFL Matchups information for opening game odds, closing lines, betting trends and ATS results for the 2020 Pro football season from VegasInsider">\r\n        \r\n        <meta name="robots"                 content="index,follow">\r\n        \r\n        <meta http-equiv="Content-Type" \tcontent="text/html; charset=ISO-8859-1">\r\n    \r\n<meta name="viewport" content="width=device-width, intial-scale=

In [6]:
client = MongoClient('localhost', 27017)
db = client.sports_betting
pages = db.pages

pages.insert_one({'html': r.content})

In [7]:
soup = BeautifulSoup(r.content, "html")

In [9]:
print(soup.prettify())

<!-- manual generation from poegenerators:poeMatchupsIndexPage Tue Aug 19 8:05 AM-->
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<html lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <title>
   NFL Matchups 2020 - Lines, Betting Trends, and ATS results
  </title>
  <meta content="IE=edge" http-equiv="x-ua-compatible"/>
  <meta content="nfl matchups,football matchups,pro football matchups" name="Keywords"/>
  <meta content="Get top NFL Matchups information for opening game odds, closing lines, betting trends and ATS results for the 2020 Pro football season from VegasInsider" name="Description"/>
  <meta content="index,follow" name="robots"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, intial-scale=1.0" name="viewport"/>
  <link href="/include/default.css?v=4" rel="Stylesheet" type="TEXT/CSS"/>
  <link href="/include/scratchpad.css?v=4" rel=

In [10]:
print(soup.title)

<title>NFL Matchups 2020 - Lines, Betting Trends, and ATS results</title>


In [40]:
divs = soup.findAll("div", {"class": "SLTables1"})

In [42]:
len(divs)

19

In [43]:
tables = [] # list accumulator for tables
for div in divs[2:]: #skipping first 2 divs since these are not specific to games
    table = div.find("table")
    tables.append(table)

In [45]:
len(tables)

17

In [79]:
tables[0]

<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tr>
<td class="viHeaderNorm">
								Houston Texans @ Kansas City Chiefs
							</td>
</tr>
<tr>
<td class="viBodyBorderNorm">
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tr>
<td class="viSubHeader1 cellBorderL1 headerTextHot padLeft" width="30%">8:20 PM Game Time</td>
<td class="viSubHeader1 cellBorderL1 headerTextNorm padCenter" colspan="2" width="20%">Record</td>
<td class="viSubHeader1 cellBorderL1 headerTextNorm padCenter" colspan="2" width="20%">Line</td>
<td class="viSubHeader1 cellBorderL1 headerTextNorm padCenter" colspan="3" width="20%">Betting Trend</td>
<td class="viSubHeader1 cellBorderR1 headerTextNorm padLeft" width="10%"> </td>
</tr>
<tr>
<td class="viSubHeader2 cellBorderL2 headerTextNorm padLeft">Teams</td>
<td class="viSubHeader2 cellBorderL2 headerTextNorm padLeft" nowrap="nowrap" width="14%">Win-Loss</td>
<td class="viSubHeader2 cellBorderL2 headerTextNorm padLeft" width="7%">Str

In [94]:
all_rows = []

empty_row = {
    "Team": None, "Win-Loss": None, "Streak": None, "Open": None, "Closing": None, "Consensus_Spread": None,
    "Consensus_Money": None,"Consensus_OU": None
}

for table in tables:
    rows = table.find_all("tr")
    for row in rows[4:6]:
        new_row = copy.copy(empty_row)
        # A list of all the entries in the row.
        columns = row.find_all("td")
        new_row['Team'] = columns[0].text.strip()
        new_row['Win-Loss'] = columns[1].text.strip()
        new_row['Streak'] = columns[2].text.strip()
        new_row['Open'] = columns[3].text.strip()
        new_row['Closing'] = columns[4].text.strip()
        new_row['Consensus_Spread'] = columns[5].text.strip()
        new_row['Consensus_Money'] = columns[6].text.strip()
        new_row['Consensus_OU'] = columns[7].text.strip()
        all_rows.append(new_row)  
    

In [96]:
len(all_rows)

34

In [98]:
all_rows

[{'Team': '451 Houston',
  'Win-Loss': '(0-0) (0-0 A)',
  'Streak': '',
  'Open': '56.5',
  'Closing': '53.5',
  'Consensus_Spread': '50%',
  'Consensus_Money': '26%',
  'Consensus_OU': '72%'},
 {'Team': '452 Kansas City\xa0Â«',
  'Win-Loss': '(0-0) (0-0 H)',
  'Streak': '',
  'Open': '-10',
  'Closing': '-9.5',
  'Consensus_Spread': '50%',
  'Consensus_Money': '74%',
  'Consensus_OU': '28%'},
 {'Team': '0 Las Vegas\xa0Â«',
  'Win-Loss': '(0-0) (0-0 A)',
  'Streak': '',
  'Open': '',
  'Closing': '',
  'Consensus_Spread': '',
  'Consensus_Money': '',
  'Consensus_OU': ''},
 {'Team': '0 Carolina',
  'Win-Loss': '(0-0) (0-0 H)',
  'Streak': '',
  'Open': '',
  'Closing': '',
  'Consensus_Spread': '',
  'Consensus_Money': '',
  'Consensus_OU': ''},
 {'Team': '453 Miami',
  'Win-Loss': '(0-0) (0-0 A)',
  'Streak': '',
  'Open': '43.5',
  'Closing': '41.5',
  'Consensus_Spread': '42%',
  'Consensus_Money': '51%',
  'Consensus_OU': '58%'},
 {'Team': '454 New England\xa0Â«',
  'Win-Loss': '(0

In [100]:
db = client.sports_betting
nfl_lines = db.nfl_lines

In [101]:
for row in all_rows:
    nfl_lines.insert_one(row)

In [105]:
nfl_rows = nfl_lines.find()
nfl_pddf = pd.DataFrame(list(nfl_rows))

In [106]:
nfl_pddf

,_id,Team,Win-Loss,Streak,Open,Closing,Consensus_Spread,Consensus_Money,Consensus_OU
0,5f984e6070b745e0f68c0030,451 Houston,(0-0) (0-0 A),,56.5,53.5,50%,26%,72%
1,5f984e6070b745e0f68c0031,452 Kansas City Â«,(0-0) (0-0 H),,-10,-9.5,50%,74%,28%
2,5f984e6070b745e0f68c0032,0 Las Vegas Â«,(0-0) (0-0 A),,,,,,
3,5f984e6070b745e0f68c0033,0 Carolina,(0-0) (0-0 H),,,,,,
4,5f984e6070b745e0f68c0034,453 Miami,(0-0) (0-0 A),,43.5,41.5,42%,51%,58%
5,5f984e6070b745e0f68c0035,454 New England Â«,(0-0) (0-0 H),,-5.5,-7,58%,49%,42%
6,5f984e6070b745e0f68c0036,455 Cleveland,(0-0) (0-0 A),,48.5,47.5,40%,25%,72%
7,5f984e6070b745e0f68c0037,456 Baltimore Â«,(0-0) (0-0 H),,-9,-7,60%,75%,28%
8,5f984e6070b745e0f68c0038,457 N.Y. Jets,(0-0) (0-0 A),,40.0,39,20%,23%,49%
9,5f984e6070b745e0f68c0039,458 Buffalo Â«,(0-0) (0-0 H),,-5.5,-6,80%,77%,51%


In [109]:
test_url2 = 'https://www.vegasinsider.com/nfl/matchups/matchups.cfm/week/2/season/2020'
r = requests.get(test_url2)

In [110]:
r.status_code

200

In [111]:
r.content

b'<!-- manual generation from poegenerators:poeMatchupsIndexPage Tue Aug 19 8:05 AM-->\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN"> \r\n<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/" lang="en">\r\n<head>\r\n\r\n        <title>NFL Matchups 2020 - Lines, Betting Trends, and ATS results</title>\r\n        <meta http-equiv="x-ua-compatible" \tcontent="IE=edge">\r\n        <meta name="Keywords" \t\t\t\tcontent="nfl matchups,football matchups,pro football matchups">\r\n        <meta name="Description" \t\t\tcontent="Get top NFL Matchups information for opening game odds, closing lines, betting trends and ATS results for the 2020 Pro football season from VegasInsider">\r\n        \r\n        <meta name="robots"                 content="index,follow">\r\n        \r\n        <meta http-equiv="Content-Type" \tcontent="text/html; charset=ISO-8859-1">\r\n    \r\n<meta name="viewport" content="width=device-width, intial-scale=

In [112]:
pages.insert_one({'html': r.content})

In [113]:
soup = BeautifulSoup(r.content, "html")

In [114]:
divs = soup.findAll("div", {"class": "SLTables1"})

tables = [] # list accumulator for tables
for div in divs[2:]: #skipping first 2 divs since these are not specific to games
    table = div.find("table")
    tables.append(table)
    
all_rows = []

empty_row = {
    "Team": None, "Win-Loss": None, "Streak": None, "Open": None, "Closing": None, "Consensus_Spread": None,
    "Consensus_Money": None,"Consensus_OU": None
}

for table in tables:
    rows = table.find_all("tr")
    for row in rows[4:6]:
        new_row = copy.copy(empty_row)
        # A list of all the entries in the row.
        columns = row.find_all("td")
        new_row['Team'] = columns[0].text.strip()
        new_row['Win-Loss'] = columns[1].text.strip()
        new_row['Streak'] = columns[2].text.strip()
        new_row['Open'] = columns[3].text.strip()
        new_row['Closing'] = columns[4].text.strip()
        new_row['Consensus_Spread'] = columns[5].text.strip()
        new_row['Consensus_Money'] = columns[6].text.strip()
        new_row['Consensus_OU'] = columns[7].text.strip()
        all_rows.append(new_row)  

In [115]:
all_rows

[{'Team': '101 Cincinnati',
  'Win-Loss': '(0-1) (0-0 A)',
  'Streak': 'L-1',
  'Open': '46.0',
  'Closing': '44.5',
  'Consensus_Spread': '58%',
  'Consensus_Money': '42%',
  'Consensus_OU': '59%'},
 {'Team': '102 Cleveland\xa0Â«',
  'Win-Loss': '(0-1) (0-0 H)',
  'Streak': 'L-1',
  'Open': '-8.5',
  'Closing': '-6',
  'Consensus_Spread': '42%',
  'Consensus_Money': '58%',
  'Consensus_OU': '41%'},
 {'Team': '261 L.A. Rams\xa0Â«',
  'Win-Loss': '(1-0) (0-0 A)',
  'Streak': 'W-1',
  'Open': '49.0',
  'Closing': '46',
  'Consensus_Spread': '70%',
  'Consensus_Money': '56%',
  'Consensus_OU': '75%'},
 {'Team': '262 Philadelphia',
  'Win-Loss': '(0-1) (0-0 H)',
  'Streak': 'L-1',
  'Open': '-4.5',
  'Closing': '-1.5',
  'Consensus_Spread': '30%',
  'Consensus_Money': '44%',
  'Consensus_OU': '25%'},
 {'Team': '263 Carolina',
  'Win-Loss': '(0-1) (0-0 A)',
  'Streak': 'L-1',
  'Open': '48.0',
  'Closing': '47',
  'Consensus_Spread': '39%',
  'Consensus_Money': '21%',
  'Consensus_OU': '74%

In [116]:
len(all_rows)

32

In [117]:
for row in all_rows:
    nfl_lines.insert_one(row)

In [118]:
nfl_rows = nfl_lines.find()
nfl_pddf = pd.DataFrame(list(nfl_rows))

In [119]:
nfl_pddf

,_id,Team,Win-Loss,Streak,Open,Closing,Consensus_Spread,Consensus_Money,Consensus_OU
0,5f984e6070b745e0f68c0030,451 Houston,(0-0) (0-0 A),,56.5,53.5,50%,26%,72%
1,5f984e6070b745e0f68c0031,452 Kansas City Â«,(0-0) (0-0 H),,-10,-9.5,50%,74%,28%
2,5f984e6070b745e0f68c0032,0 Las Vegas Â«,(0-0) (0-0 A),,,,,,
3,5f984e6070b745e0f68c0033,0 Carolina,(0-0) (0-0 H),,,,,,
4,5f984e6070b745e0f68c0034,453 Miami,(0-0) (0-0 A),,43.5,41.5,42%,51%,58%
...,...,...,...,...,...,...,...,...,...
61,5f9853fe70b745e0f68c006e,286 L.A. Chargers,(1-0) (0-0 H),W-1,51.5,47.5,17%,10%,36%
62,5f9853fe70b745e0f68c006f,287 New England,(1-0) (0-0 A),W-1,44.0,45,35%,17%,63%
63,5f9853fe70b745e0f68c0070,288 Seattle Â«,(1-0) (0-0 H),W-1,-3.5,-4.5,65%,83%,37%
64,5f9853fe70b745e0f68c0071,289 New Orleans,(1-0) (0-0 A),W-1,-5,-4,58%,60%,70%


In [153]:
db = client.sports_betting_capstone
nfl_lines = db.nfl_lines

In [154]:
nfl_rows = nfl_lines.find()

In [155]:
nfl_pddf = pd.DataFrame(list(nfl_rows))

In [158]:
nfl_pddf.drop(['_id', 'Win-Loss', 'Streak', axis = 1])

SyntaxError: invalid syntax (<ipython-input-158-a843c961f8b3>, line 1)

In [157]:
nfl_pddf.to_csv('output.csv')

In [159]:
client = MongoClient('localhost', 27017)
db = client.sports_betting_capstone
pages = db.pages
nfl_results = db.nfl_results

In [160]:
url_nfl_1 = 'https://www.footballdb.com/scores/index.html?lg=NFL&yr='
url_nfl_2 = '&type=reg&wk='

In [178]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}

for year in range(2016,2020):
    for week in range(1,18):
        r = requests.get(url_nfl_1 + str(year) + url_nfl_2 + str(week), headers = headers)
        pages.insert_one({'html': r.content})

        soup = BeautifulSoup(r.content, "html")

        divs = soup.findAll("div", {"class": "lngame"})

        tables = [] # list accumulator for tables

        for div in divs:
            table = div.find("table")
            tables.append(table)

        all_rows = []

        empty_row = {
            "Team": None, "Q1": None, "Q2": None, "Q3": None, "Q4": None, "Final": None,
            "Season": year, "Week": week
        }

        for table in tables:
            rows = table.find_all("tr")
            for row in rows[1:]:
                new_row = copy.copy(empty_row)
                # A list of all the entries in the row.
                columns = row.find_all("td")
                new_row['Team'] = columns[0].text.strip()
                new_row['Q1'] = columns[1].text.strip()
                new_row['Q2'] = columns[2].text.strip()
                new_row['Q3'] = columns[3].text.strip()
                new_row['Q4'] = columns[4].text.strip()
                new_row['Final'] = columns[5].text.strip()
                all_rows.append(new_row)

        for row in all_rows:
            nfl_results.insert_one(row)

IndexError: list index out of range

In [166]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}
r = requests.get('https://www.footballdb.com/scores/index.html?lg=NFL&yr=2020&type=reg&wk=1', headers = headers)

In [167]:
r.status_code

200

In [168]:
r.content

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<meta name="viewport" content="initial-scale=1.0, width=device-width" />\n<title>2020 NFL Scores - Week 1 | The Football Database</title>\n<meta name="description" content="View scores and results from week 1 of the 2020 NFL Regular Season" />\n<meta name="robots" content="index,follow" />\n<meta property="og:site_name" content="FootballDB.com">\n<meta property="og:title" content="2020 NFL Scores - Week 1 | The Football Database" />\n<meta property="og:type" content="website" />\n<meta property="og:description" content="View scores and results from week 1 of the 2020 NFL Regular Season" />\n<meta property="og:image" content="https://www.footballdb.com/images/footballdb_200x200.png" />\n<meta property="og:url" content="https://www.footballdb.com/scores/index.html?lg=NFL&yr=2020&type=reg&wk=1" />\n<meta name="twitter:card" content="summary" />\n<meta name="twitt

In [190]:
nfl_rows = nfl_results.find()

In [191]:
nfl_pddf = pd.DataFrame(list(nfl_rows))

In [192]:
nfl_pddf

,_id,Team,Final,Season,Week
0,5f993e2bd6f3f1cd63a4cc57,Oakland (12-4),14,2016,18
1,5f993e2bd6f3f1cd63a4cc58,Houston (9-7),27,2016,18
2,5f993e2bd6f3f1cd63a4cc59,Detroit (9-7),6,2016,18
3,5f993e2bd6f3f1cd63a4cc5a,Seattle (10-5-1),26,2016,18
4,5f993e2bd6f3f1cd63a4cc5b,Miami (10-6),12,2016,18
...,...,...,...,...,...
2341,5f993f6e0a7014855df1ea2c,LA Chargers (2-4),39,2020,7
2342,5f993f6e0a7014855df1ea2d,Seattle (5-1),34,2020,7
2343,5f993f6e0a7014855df1ea2e,Arizona (5-2),37,2020,7
2344,5f993f6e0a7014855df1ea2f,Chicago (5-2),10,2020,7


In [193]:
nfl_pddf.to_csv('results.csv')